In [ ]:
import numpy as np
import arms
import matplotlib.pyplot as plt

# Multi-Armed Bandits
A MAB problem can be interpreted as a Markov Decision Process (MDP) with a single state and multiple actions (i.e., arms). At each round, the learning agent executes an action (i.e., arm) and observes a random realization of the reward associated to the arm. The goal is to find the arm with the highest mean reward

A MAB problem is defined by $N$ arms. Each arm $i$ has an associated reward distribution $\Lambda_i$ with mean $\mu_i$.

Let's define our first bandit problem

In [ ]:
# Build your own bandit problem
arm1 = arms.ArmBernoulli(0.30)
arm2 = arms.ArmBernoulli(0.45)
arm3 = arms.ArmBernoulli(0.15)
arm4 = arms.ArmBernoulli(0.10)

MAB = [arm1, arm2, arm3, arm4]

In [ ]:
nb_arms = len(MAB)
means = [el.mean for el in MAB]

# Display the means of your bandit (to find the best)
print('means: {}'.format(means))
mu_max = np.max(means)
print('best arm: {}'.format(mu_max))

# Regret
We evaluate exploration-exploitation algorithms based on the expected regret.
The **expected regret** measures the difference between executing the optimal arm $a^\star$ and following the learning agent:
$$R(T) = \mathbb{E}\Big[\sum_{t=1}^T \mu^\star - r_t \Big]$$

# Explore-then-commit
ETC algorithms are composed by two phases. In the first phase, we do pure exploration in order to identify the problem (for example we execute a random policy). In the second phase, we act greedily based on the estimates.

1. exploration phase
At each time $t \in [0, T_e]$, we select a random action $a_t \sim \mathcal{U}(N)$ and observe a reward $r_t \sim \Lambda_{a_t}$.

We use this phase to build an estimate of the mean reward of each arm
$$ \hat{\mu}_i = \frac{1}{N_{T_e}(i)} \sum_{t=0}^{T_e} r_t \cdot 1(a_t = i)$$
where $N_{T_e}(i)$ is the number of times arm $i$ was played.

2. commit
From $T_e$ on, we select $a_t = \arg\max_{i \in [N]} \hat{\mu}_i$


In [ ]:
Te = 200 # length of the pure exploration phase
Tc = 200 # length of the commit phase
T = Te + Tc

regret = np.zeros((T,))

cum_R = ...  # cumulative reward for each arm
N = ...      # number of pulls for each arm
for t in range(Te):
    a = ...
    reward = MAB[a].sample()
    
    # update statistics
    N[a] += ...
    cum_R[a] += ... 
    
    # update regret (instantaneous regret)
    regret[t] = ...
    
mu_hat = ...
bestarm_hat = ...
for t in range(Te, Te+Tc):
    reward = MAB[bestarm_hat].sample()
    
    # update regret
    regret[t] = ...

# we want the cumulative regret
cum_regret = np.cumsum(regret)

plt.plot(cum_regret)
    
    

## What is wrong with that?

Algorithm and environment (MAB problem) are stochastics. We want to have meaningful results, thus we average over multiple runs.

In [ ]:
nb_repetitions = 40

regret = np.zeros((nb_repetitions, T))

for it in range(nb_repetitions):
    ## here you should copy previous code
    ## and change the upate of the regret

cum_regret = np.cumsum(regret, axis=1)
mean_regret = cum_regret.mean(axis=0)
std = cum_regret.std(axis=0) / np.sqrt(nb_repetitions)

plt.plot(mean_regret, label="ETC")
plt.fill_between(np.arange(T), mean_regret + std, mean_regret - std, alpha=0.1)
plt.legend()

# save current regret
regret_ETC = mean_regret
std_ETC= std
    

# $\epsilon$-Greedy
This algorithm simply builds an estimate $\hat{\mu}_i$ of the mean reward of each arm and at each round it selects the action accorging to the following policy
$$a_t = \begin{cases}
\mathcal{U}(N) & w.p.~\epsilon\\
\arg\max_{i} \hat{\mu}_i & w.p.~1-\epsilon
\end{cases}$$

In [ ]:
regret = np.zeros((nb_repetitions, T))
epsilon = 0.1

for it in range(nb_repetitions):
    ...
    
cum_regret = np.cumsum(regret, axis=1)
mean_regret = cum_regret.mean(axis=0)
std = cum_regret.std(axis=0) / np.sqrt(nb_repetitions)

plt.plot(mean_regret, label="eps-greedy")
plt.fill_between(np.arange(T), mean_regret + std, mean_regret - std, alpha=0.1)
plt.legend()


# save current regret
regret_EPSGREEDY= mean_regret
std_EPSGREEDY= std


# The UCB1 algorithm

The UCB1 algorithm is proposed by [Auer et al](https://homes.di.unimi.it/~cesabian/Pubblicazioni/ml-02.pdf) for bandit instances with bounded rewards (in $[0,1]$ to fix the ideas). One can extend it to depend on some parameter $\alpha$: 

$$A_{t} = \underset{a}{\text{argmax}} \left[\hat{\mu}_a(t-1) + \sqrt{\frac{\alpha \log(t)}{N_a(t-1)}}\right],$$
where $\hat{\mu}_a(t)$ is the empirical mean of arm $a$ after $t$ rounds and $N_a(t)$ is the number of selections of arm $a$ till that time. 

UCB1 was originally proposed with $\alpha = 2$. Its analysis was later refined to allow for $\alpha > 1/2$ (see [here](http://sbubeck.com/Bubeckthesis.pdf) or [here](https://hal.archives-ouvertes.fr/hal-00738209/file/klucb.pdf)).

* Implement UCB($\alpha$).


In [ ]:
regret = np.zeros((nb_repetitions, T))
alpha = 0.5


for it in range(nb_repetitions):
    ...
    
cum_regret = np.cumsum(regret, axis=1)
mean_regret = cum_regret.mean(axis=0)
std = cum_regret.std(axis=0) / np.sqrt(nb_repetitions)

plt.plot(mean_regret, label="UCB")
plt.fill_between(np.arange(T), mean_regret + std, mean_regret - std, alpha=0.1)
plt.legend()


# save current regret
regret_UCB = mean_regret
std_UCB= std

Plot all the algorithms

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(regret_ETC, label="ETC")
plt.fill_between(np.arange(T), regret_UCB + std_UCB, regret_UCB - std_UCB, alpha=0.1)
plt.plot(regret_EPSGREEDY, label="EPS GREEDY")
plt.fill_between(np.arange(T), regret_UCB + std_UCB, regret_UCB - std_UCB, alpha=0.1)
plt.plot(regret_UCB, label="UCB")
plt.fill_between(np.arange(T), regret_UCB + std_UCB, regret_UCB - std_UCB, alpha=0.1)
plt.legend()